### Preprocessing the Data for a Neural Network

In [158]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
import pandas as pd
import tensorflow as tf
from keras.utils import np_utils
from config import db_password
import psycopg2

In [159]:
# Import and read csv
# import pandas as pd 
# kushy_washington_cleaned_df = pd.read_csv("./Data/kushy_washington_cleaned.csv")
# print(kushy_washington_cleaned_df.shape)
# kushy_washington_cleaned_df.head()

In [160]:
# Connect to sql database
con = psycopg2.connect(database='Medical-Marijuana-Group', user = 'postgres', password = db_password, host = '127.0.0.1', port = '5432')

# Create a new cursor
cur = con.cursor()

print('Opened database successfully')

Opened database successfully


In [161]:
# Create function that takes in a PostgreSQL query and outputs a Pandas dataframe
def create_pandas_df(sql_query, database = con):
    df = pd.read_sql_query(sql_query, database)
    return df
  
# Utilize the create_pandas_df function to create a Pandas dataframe
# Store the data as a variable
kushy_washington_cleaned_df = create_pandas_df('SELECT * FROM kushy_washington_cleaned')
print(kushy_washington_cleaned_df.shape)
kushy_washington_cleaned_df.head()

(722, 36)


,test_id,test_strain,type,ailment_1,ailment_2,ailment_3,ailment_4,ailment_5,effects_1,effects_2,...,inventory_type,chemotype,strain_leafly_page_rank,strain_leafly_review_rank,strain_tests,strain_chemotype,strain_prop_chemotype1,strain_prop_chemotype2,strain_prop_chemotype3,strain_modal_chemo_prop
0,19,Dutch Hawaiian,Sativa,Depression,Stress,Pain,Nausea,None,Happy,Energetic,...,Flower Lot,1,619,141,425,1,1.0,0.0,0.0,1.0
1,21,SleeStack,Hybrid,Stress,Depression,None,None,None,Uplifted,Euphoric,...,Flower Lot,1,1276,151,63,1,1.0,0.0,0.0,1.0
2,22,Space Needle,Sativa,Depression,Stress,None,None,None,Uplifted,Euphoric,...,Flower Lot,1,0,0,0,0,0.0,0.0,0.0,0.0
3,23,Double Purple Doja,Hybrid,Pain,Stress,Inflammation,Muscle Spasms,None,Relaxed,Creative,...,Flower Lot,1,1115,151,52,1,1.0,0.0,0.0,1.0
4,37,Jack Skellington,Hybrid,Stress,Depression,Pain,Muscle Spasms,None,Creative,Relaxed,...,Flower Lot,1,708,141,124,1,1.0,0.0,0.0,1.0


In [162]:
# Close the cursor and connection so the server can allocate bandwidth to other requests
cur.close()
con.close()
print('Closed database successfully')

Closed database successfully


In [163]:
# Drop non-beneficial identification columns
kushy_washington_df = kushy_washington_cleaned_df.drop(['test_id', 'test_strain'], axis=1)
print(kushy_washington_df.shape)
kushy_washington_df.head()

(722, 34)


,type,ailment_1,ailment_2,ailment_3,ailment_4,ailment_5,effects_1,effects_2,effects_3,effects_4,...,inventory_type,chemotype,strain_leafly_page_rank,strain_leafly_review_rank,strain_tests,strain_chemotype,strain_prop_chemotype1,strain_prop_chemotype2,strain_prop_chemotype3,strain_modal_chemo_prop
0,Sativa,Depression,Stress,Pain,Nausea,None,Happy,Energetic,Uplifted,Euphoric,...,Flower Lot,1,619,141,425,1,1.0,0.0,0.0,1.0
1,Hybrid,Stress,Depression,None,None,None,Uplifted,Euphoric,Creative,Relaxed,...,Flower Lot,1,1276,151,63,1,1.0,0.0,0.0,1.0
2,Sativa,Depression,Stress,None,None,None,Uplifted,Euphoric,Happy,Hungry,...,Flower Lot,1,0,0,0,0,0.0,0.0,0.0,0.0
3,Hybrid,Pain,Stress,Inflammation,Muscle Spasms,None,Relaxed,Creative,Sleepy,Happy,...,Flower Lot,1,1115,151,52,1,1.0,0.0,0.0,1.0
4,Hybrid,Stress,Depression,Pain,Muscle Spasms,None,Creative,Relaxed,Euphoric,Energetic,...,Flower Lot,1,708,141,124,1,1.0,0.0,0.0,1.0


In [164]:
# Determine number of unique values in each column
kushy_washington_df.nunique()

type                           4
ailment_1                      9
ailment_2                      9
ailment_3                     10
ailment_4                     10
ailment_5                      9
effects_1                     14
effects_2                     23
effects_3                     24
effects_4                     26
effects_5                     28
effects_6                      4
effects_7                      4
effects_8                      4
flavor_1                      32
flavor_2                      50
flavor_3                      42
ailment_count                  6
effects_count                  9
flavor_count                   4
thc_max                      697
cbd_max                      207
thc_cbd                      558
cbd_thc                       48
inventory_type                 7
chemotype                      3
strain_leafly_page_rank      640
strain_leafly_review_rank    128
strain_tests                 291
strain_chemotype               4
strain_pro

In [165]:
# Look at value counts for binning
# [column]_counts = kushy_washington_df.[column].value_counts()
# [oolumn]_counts

In [166]:
# Visualize the value counts
# [column]_counts.plot.density()

In [167]:
# Determine which values to replace if counts are less than ...?
# replace_application = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
# for app in replace_application:
#    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
#application_df.APPLICATION_TYPE.value_counts()

In [168]:
# Look at CLASSIFICATION value counts for binning
# classification_counts = application_df.CLASSIFICATION.value_counts()
# classification_counts

In [169]:
# Visualize the value counts of CLASSIFICATION
# classification_counts.plot.density()

In [170]:
# Determine which values to replace if counts are less than ..?
# replace_class = list(classification_counts[classification_counts < 1883].index)

# Replace in dataframe
# for cls in replace_class:
#    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
# application_df.CLASSIFICATION.value_counts()

In [171]:
# Generate our categorical variable lists
# application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
# application_cat

In [172]:
# Create a OneHotEncoder instance
# enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
# encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
# encode_df.columns = enc.get_feature_names_out(application_cat)
# encode_df.head()

In [173]:
# Merge one-hot encoded features and drop the originals
# application_df = application_df.merge(encode_df,left_index=True, right_index=True)
# application_df = application_df.drop(application_cat,axis=1)
# application_df.head()

In [174]:
# Determine number of unique values in each column
kushy_washington_df.dtypes

type                          object
ailment_1                     object
ailment_2                     object
ailment_3                     object
ailment_4                     object
ailment_5                     object
effects_1                     object
effects_2                     object
effects_3                     object
effects_4                     object
effects_5                     object
effects_6                     object
effects_7                     object
effects_8                     object
flavor_1                      object
flavor_2                      object
flavor_3                      object
ailment_count                  int64
effects_count                  int64
flavor_count                   int64
thc_max                      float64
cbd_max                      float64
thc_cbd                      float64
cbd_thc                      float64
inventory_type                object
chemotype                      int64
strain_leafly_page_rank        int64
s

In [175]:
# Use labelencoder to encode target ailment_1 column and check datatype
le = LabelEncoder()
kushy_washington_df['ailment_1'] = le.fit_transform(kushy_washington_df['ailment_1'])
kushy_washington_df['ailment_1']

0      0
1      8
2      0
3      7
4      8
      ..
717    7
718    1
719    8
720    6
721    6
Name: ailment_1, Length: 722, dtype: int32

In [204]:
# Check datatypes
kushy_washington_df.dtypes

type                          object
ailment_1                      int32
ailment_2                     object
ailment_3                     object
ailment_4                     object
ailment_5                     object
effects_1                     object
effects_2                     object
effects_3                     object
effects_4                     object
effects_5                     object
effects_6                     object
effects_7                     object
effects_8                     object
flavor_1                      object
flavor_2                      object
flavor_3                      object
ailment_count                  int64
effects_count                  int64
flavor_count                   int64
thc_max                      float64
cbd_max                      float64
thc_cbd                      float64
cbd_thc                      float64
inventory_type                object
chemotype                      int64
strain_leafly_page_rank        int64
s

In [206]:
# Convert vector of integers to one hot encoding using keras function to_categorical
dummy_y = np_utils.to_categorical(kushy_washington_df['ailment_1'])
print(dummy_y.shape)

(722, 9)


In [194]:
# Use get_dummies() to encode object columns
kushy_washington_df_encoded = pd.get_dummies(kushy_washington_df, columns=None)
print(kushy_washington_df_encoded.shape)
kushy_washington_df_encoded.head()

(722, 317)


,ailment_1,ailment_count,effects_count,flavor_count,thc_max,cbd_max,thc_cbd,cbd_thc,chemotype,strain_leafly_page_rank,...,flavor_3_Tree,flavor_3_Tropical,flavor_3_Woody,inventory_type_Bubble Hash,inventory_type_CO2 Hash Oil,inventory_type_Flower Lot,inventory_type_Food Grade Solvent Extract,inventory_type_Hash,inventory_type_Hydrocarbon Wax,inventory_type_Marijuana Mix
0,0,4,7,2,20.1309,0.54,37.28,0.03,1,619,...,0,0,0,0,0,1,0,0,0,0
1,8,2,7,2,9.9139,0.00,0.00,0.00,1,1276,...,0,0,0,0,0,1,0,0,0,0
2,0,2,7,1,10.8132,0.00,0.00,0.00,1,0,...,0,0,0,0,0,1,0,0,0,0
3,7,4,7,2,11.6702,0.00,0.00,0.00,1,1115,...,0,0,0,0,0,1,0,0,0,0
4,8,4,8,2,13.4657,0.00,0.00,0.00,1,708,...,0,0,0,0,0,1,0,0,0,0


In [210]:
# Split preprocessed data into features and target arrays
X = kushy_washington_df_encoded.drop(['ailment_1'],axis=1).values
y = dummy_y
# Split preprocessed data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=72)

In [211]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train and Evaluate the Model

In [212]:
# Define the model - deep neural net, i.e., the number of input/output features and hidden nodes for each layer
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 160
hidden_nodes_layer2 = 80

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=9, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 160)               50720     
                                                                 
 dense_31 (Dense)            (None, 80)                12880     
                                                                 
 dense_32 (Dense)            (None, 9)                 729       
                                                                 
Total params: 64,329
Trainable params: 64,329
Non-trainable params: 0
_________________________________________________________________


In [216]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"
batch_size = 17

# Compile the model
nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=20*batch_size)

In [217]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=1000,callbacks=[cp_callback])

Epoch 1/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.9201e-04 - accuracy: 1.0000
Epoch 2/1000
17/17 [==============================] - 0s 2ms/step - loss: 7.6313e-05 - accuracy: 1.0000
Epoch 3/1000
17/17 [==============================] - 0s 2ms/step - loss: 2.1367e-05 - accuracy: 1.0000
Epoch 4/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.1635e-05 - accuracy: 1.0000
Epoch 5/1000
17/17 [==============================] - 0s 2ms/step - loss: 6.6046e-06 - accuracy: 1.0000
Epoch 6/1000
17/17 [==============================] - 0s 2ms/step - loss: 5.0673e-06 - accuracy: 1.0000
Epoch 7/1000
17/17 [==============================] - 0s 2ms/step - loss: 4.1996e-06 - accuracy: 1.0000
Epoch 8/1000
17/17 [==============================] - 0s 2ms/step - loss: 3.5782e-06 - accuracy: 1.0000
Epoch 9/1000
17/17 [==============================] - 0s 2ms/step - loss: 3.1406e-06 - accuracy: 1.0000
Epoch 10/1000
17/17 [==============================] - 0s 2ms/st

17/17 [==============================] - 0s 2ms/step - loss: 1.3309e-07 - accuracy: 1.0000
Epoch 76/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.3023e-07 - accuracy: 1.0000
Epoch 77/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.2802e-07 - accuracy: 1.0000
Epoch 78/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.2384e-07 - accuracy: 1.0000
Epoch 79/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.2097e-07 - accuracy: 1.0000
Epoch 80/1000
 1/17 [>.............................] - ETA: 0s - loss: 1.4901e-07 - accuracy: 1.0000
Epoch 80: saving model to checkpoints\weights.80.hdf5
17/17 [==============================] - 0s 4ms/step - loss: 1.1723e-07 - accuracy: 1.0000
Epoch 81/1000
17/17 [==============================] - 0s 3ms/step - loss: 1.1480e-07 - accuracy: 1.0000
Epoch 82/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.1150e-07 - accuracy: 1.0000
Epoch 83/1000
17/17 [============

17/17 [==============================] - 0s 3ms/step - loss: 2.9747e-08 - accuracy: 1.0000
Epoch 148/1000
17/17 [==============================] - 0s 3ms/step - loss: 2.8425e-08 - accuracy: 1.0000
Epoch 149/1000
17/17 [==============================] - 0s 3ms/step - loss: 2.7323e-08 - accuracy: 1.0000
Epoch 150/1000
17/17 [==============================] - 0s 2ms/step - loss: 2.6883e-08 - accuracy: 1.0000
Epoch 151/1000
17/17 [==============================] - 0s 2ms/step - loss: 2.6662e-08 - accuracy: 1.0000
Epoch 152/1000
17/17 [==============================] - 0s 3ms/step - loss: 2.6442e-08 - accuracy: 1.0000
Epoch 153/1000
17/17 [==============================] - 0s 3ms/step - loss: 2.5561e-08 - accuracy: 1.0000
Epoch 154/1000
17/17 [==============================] - 0s 2ms/step - loss: 2.5120e-08 - accuracy: 1.0000
Epoch 155/1000
17/17 [==============================] - 0s 3ms/step - loss: 2.4679e-08 - accuracy: 1.0000
Epoch 156/1000
17/17 [==============================] - 0s 2m

 1/17 [>.............................] - ETA: 0s - loss: 7.4506e-09 - accuracy: 1.0000
Epoch 220: saving model to checkpoints\weights.220.hdf5
17/17 [==============================] - 0s 4ms/step - loss: 7.4919e-09 - accuracy: 1.0000
Epoch 221/1000
17/17 [==============================] - 0s 2ms/step - loss: 7.4919e-09 - accuracy: 1.0000
Epoch 222/1000
17/17 [==============================] - 0s 2ms/step - loss: 7.4919e-09 - accuracy: 1.0000
Epoch 223/1000
17/17 [==============================] - 0s 2ms/step - loss: 7.2715e-09 - accuracy: 1.0000
Epoch 224/1000
17/17 [==============================] - 0s 2ms/step - loss: 7.0512e-09 - accuracy: 1.0000
Epoch 225/1000
17/17 [==============================] - 0s 2ms/step - loss: 7.0512e-09 - accuracy: 1.0000
Epoch 226/1000
17/17 [==============================] - 0s 2ms/step - loss: 6.8308e-09 - accuracy: 1.0000
Epoch 227/1000
17/17 [==============================] - 0s 2ms/step - loss: 6.6105e-09 - accuracy: 1.0000
Epoch 228/1000
17/17 [==

17/17 [==============================] - 0s 2ms/step - loss: 1.1017e-09 - accuracy: 1.0000
Epoch 293/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.1017e-09 - accuracy: 1.0000
Epoch 294/1000
17/17 [==============================] - 0s 2ms/step - loss: 6.6105e-10 - accuracy: 1.0000
Epoch 295/1000
17/17 [==============================] - 0s 2ms/step - loss: 6.6105e-10 - accuracy: 1.0000
Epoch 296/1000
17/17 [==============================] - 0s 2ms/step - loss: 6.6105e-10 - accuracy: 1.0000
Epoch 297/1000
17/17 [==============================] - 0s 2ms/step - loss: 6.6105e-10 - accuracy: 1.0000
Epoch 298/1000
17/17 [==============================] - 0s 2ms/step - loss: 8.8140e-10 - accuracy: 1.0000
Epoch 299/1000
17/17 [==============================] - 0s 2ms/step - loss: 6.6105e-10 - accuracy: 1.0000
Epoch 300/1000
 1/17 [>.............................] - ETA: 0s - loss: 3.7253e-09 - accuracy: 1.0000
Epoch 300: saving model to checkpoints\weights.300.hdf5
17/17 [==

...

17/17 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 655/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 656/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 657/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 658/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 659/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 660/1000
 1/17 [>.............................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 660: saving model to checkpoints\weights.660.hdf5
17/17 [==============================] - 0s 4ms/step - loss: 2.2035e-10 - accuracy: 1.0000
Epoch 661/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 662/1000
17/17 [==

17/17 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 727/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 728/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 729/1000
17/17 [==============================] - 0s 3ms/step - loss: 2.2035e-10 - accuracy: 1.0000
Epoch 730/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 731/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 732/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 733/1000
17/17 [==============================] - 0s 3ms/step - loss: 2.2035e-10 - accuracy: 1.0000
Epoch 734/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 735/1000
17/17 [==============================] - 0s 3m

17/17 [==============================] - 0s 2ms/step - loss: 6.6105e-10 - accuracy: 1.0000
Epoch 800/1000
 1/17 [>.............................] - ETA: 0s - loss: 3.7253e-09 - accuracy: 1.0000
Epoch 800: saving model to checkpoints\weights.800.hdf5
17/17 [==============================] - 0s 4ms/step - loss: 1.3221e-09 - accuracy: 1.0000
Epoch 801/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 802/1000
17/17 [==============================] - 0s 3ms/step - loss: 4.4070e-10 - accuracy: 1.0000
Epoch 803/1000
17/17 [==============================] - 0s 3ms/step - loss: 4.4070e-10 - accuracy: 1.0000
Epoch 804/1000
17/17 [==============================] - 0s 3ms/step - loss: 2.2035e-10 - accuracy: 1.0000
Epoch 805/1000
17/17 [==============================] - 0s 3ms/step - loss: 4.4070e-10 - accuracy: 1.0000
Epoch 806/1000
17/17 [==============================] - 0s 3ms/step - loss: 6.6105e-10 - accuracy: 1.0000
Epoch 807/1000
17/17 [==

17/17 [==============================] - 0s 2ms/step - loss: 1.5689e-06 - accuracy: 1.0000
Epoch 872/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.5127e-06 - accuracy: 1.0000
Epoch 873/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.4649e-06 - accuracy: 1.0000
Epoch 874/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.4235e-06 - accuracy: 1.0000
Epoch 875/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.3891e-06 - accuracy: 1.0000
Epoch 876/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.3495e-06 - accuracy: 1.0000
Epoch 877/1000
17/17 [==============================] - 0s 1ms/step - loss: 1.3114e-06 - accuracy: 1.0000
Epoch 878/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.2779e-06 - accuracy: 1.0000
Epoch 879/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.2440e-06 - accuracy: 1.0000
Epoch 880/1000
 1/17 [>.............................] - ETA: 

17/17 [==============================] - 0s 3ms/step - loss: 4.0124e-07 - accuracy: 1.0000
Epoch 944/1000
17/17 [==============================] - 0s 3ms/step - loss: 3.9618e-07 - accuracy: 1.0000
Epoch 945/1000
17/17 [==============================] - 0s 3ms/step - loss: 3.8979e-07 - accuracy: 1.0000
Epoch 946/1000
17/17 [==============================] - 0s 3ms/step - loss: 3.8472e-07 - accuracy: 1.0000
Epoch 947/1000
17/17 [==============================] - 0s 3ms/step - loss: 3.7987e-07 - accuracy: 1.0000
Epoch 948/1000
17/17 [==============================] - 0s 3ms/step - loss: 3.7436e-07 - accuracy: 1.0000
Epoch 949/1000
17/17 [==============================] - 0s 3ms/step - loss: 3.6929e-07 - accuracy: 1.0000
Epoch 950/1000
17/17 [==============================] - 0s 3ms/step - loss: 3.6357e-07 - accuracy: 1.0000
Epoch 951/1000
17/17 [==============================] - 0s 3ms/step - loss: 3.6048e-07 - accuracy: 1.0000
Epoch 952/1000
17/17 [==============================] - 0s 3m

In [219]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6/6 - 0s - loss: 4.0943 - accuracy: 0.7624 - 29ms/epoch - 5ms/step
Loss: 4.094273090362549, Accuracy: 0.7624309659004211


In [81]:
# Export our model to HDF5 file
nn.save("MMG_DLNN.h5")